### Часть 3. Классификация текстов
Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5).

1. Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте sklearn.model_selection.train_test_split для разделения множества отобранных документов на обучающее и тестовое.


In [1]:
import json

import bz2
import re

from tqdm import tqdm
import pandas as pd
%matplotlib inline
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:01, 3245.80it/s]


In [3]:
df_source = pd.DataFrame(responses)

In [5]:
number_of_cases = 500
df_good = df_source[df_source.rating_grade==5][:number_of_cases]
df_bad = df_source[df_source.rating_grade==1][:number_of_cases]

In [10]:
df = pd.concat([df_good, df_bad])

Прежде чем разделять массив на тренировочную и тестовую выборки - проведем подготовку текста

In [12]:
from string import punctuation
from nltk.corpus import stopwords

stops  = stopwords.words('russian') + list(punctuation) + ["``", "''"] + [
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
]
ru_words = re.compile("[А-Яа-я]+")

In [13]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

In [14]:
def words_only(text):
    return " ".join(ru_words.findall(text))

# def lemmatize(text, mystem=m):
#     try:
#         return "".join(m.lemmatize(text)).strip()
#     except:
#         return " "

def lemmatize(text, mystem=morph):
    try:
        return " ".join([mystem.parse(w)[0].normal_form for w in text.split(' ')]).strip()
    except:
        return " "

def remove_stopwords(text, mystopwords = stops):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

In [18]:
df.text = df.text.apply(preprocess)
df.rating_grade = df.rating_grade.apply(lambda x: 1 if x ==5 else 0)

Разделяем выборку на тренировочную и тестову.
В качестве X - обработанный текст, в качестве y - проставленная оценка (1 или 5)

In [19]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.text, df.rating_grade)

2. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных $n$-грам.

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       129
           1       0.88      0.90      0.89       121

    accuracy                           0.89       250
   macro avg       0.89      0.89      0.89       250
weighted avg       0.89      0.89      0.89       250



In [27]:
# baseline, использование униграм из слов
pipe_baseline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),
    ('clf', LogisticRegression(random_state=42, solver='liblinear'))
])
pipe_baseline.fit(x_train, y_train)
pred = pipe_baseline.predict(x_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       129
           1       0.88      0.90      0.89       121

    accuracy                           0.89       250
   macro avg       0.89      0.89      0.89       250
weighted avg       0.89      0.89      0.89       250



In [28]:
# использование 2-грам из слов
pipe_bigram = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(2, 2))),
    ('clf', LogisticRegression(random_state=42, solver='liblinear'))
])
pipe_bigram.fit(x_train, y_train)
pred = pipe_bigram.predict(x_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       123
           1       0.83      0.81      0.82       127

    accuracy                           0.82       250
   macro avg       0.82      0.82      0.82       250
weighted avg       0.82      0.82      0.82       250



In [30]:
# использование 3-грам из слов
pipe_trigram = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(3, 3))),
    ('clf', LogisticRegression(random_state=42, solver='liblinear'))
])
pipe_trigram.fit(x_train, y_train)
pred = pipe_trigram.predict(x_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.70      0.76      0.73       116
           1       0.77      0.72      0.74       134

    accuracy                           0.74       250
   macro avg       0.74      0.74      0.74       250
weighted avg       0.74      0.74      0.74       250



In [33]:
# использование символьных n-gram
pipe_char = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(3, 5), analyzer="char")),
    ('clf', LogisticRegression(random_state=42, solver='liblinear'))
])
pipe_char.fit(x_train, y_train)
pred = pipe_char.predict(x_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91       128
           1       0.90      0.92      0.91       122

    accuracy                           0.91       250
   macro avg       0.91      0.91      0.91       250
weighted avg       0.91      0.91      0.91       250



**ВЫВОД**
Самый интересный результат показало использование символьных n-gramm в диапазоне от 3 до 5 символов.


3. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:
- 1-ый вариант: $tf-idf$ преобразование (sklearn.feature_extraction.text.TfidfTransformer) и сингулярное разложение (оно же – латентый семантический анализ) (sklearn.decomposition.TruncatedSVD),
- 2-ой вариант: тематические модели LDA (sklearn.decomposition.LatentDirichletAllocation).

In [51]:
from sklearn.decomposition import TruncatedSVD
pipe_tsvd = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(3, 5), analyzer="char")),
    ('t-svd', TruncatedSVD(n_components=30, random_state=42)),
    ('clf', LogisticRegression(random_state=42, solver='liblinear'))
])
pipe_tsvd.fit(x_train, y_train)
pred = pipe_tsvd.predict(x_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       130
           1       0.87      0.90      0.89       120

    accuracy                           0.89       250
   macro avg       0.89      0.89      0.89       250
weighted avg       0.89      0.89      0.89       250



In [50]:
from sklearn.decomposition import LatentDirichletAllocation
pipe_lda = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(3, 5), analyzer="char")),
    ('t-svd', LatentDirichletAllocation(n_components=10, random_state=42, n_jobs=16)),
    ('clf', LogisticRegression(random_state=42))
])
pipe_lda.fit(x_train, y_train)
pred = pipe_lda.predict(x_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.75      0.67      0.71       140
           1       0.63      0.71      0.67       110

    accuracy                           0.69       250
   macro avg       0.69      0.69      0.69       250
weighted avg       0.69      0.69      0.69       250



**ВЫВОД**
Непродолжительные эксперименты с применением способов снижения размерности, не дали прироста в точности.
Вероятно, это связано с небольшим количеством данных на входе. Также важно произвести тюнинг гиперпараметров.
